In [1]:
import os
os.chdir("/home/atulgang/Thesis")


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

2018-05-04 15:56:43,501 : INFO : 'pattern' package not found; tag filters are not available for English


In [4]:

# WordNet only cares about 5 parts of speech.
# The other parts of speech will be tagged as nouns.

part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

wnl = WordNetLemmatizer()

def convert_tag(penn_tag):
    '''
    convert_tag() accepts the **first letter** of a Penn part-of-speech tag,
    then uses a dict lookup to convert it to the appropriate WordNet tag.
    '''
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        # other parts of speech will be tagged as nouns
        return 'n'


def tag_and_lem(element):
    '''
    tag_and_lem() accepts a token list, tags, converts tags,
    lemmatizes, and returns a string
    '''
    # list of tuples [('token', 'tag'), ('token2', 'tag2')...]
    sent = pos_tag(element) # must tag in context
    return [ wnl.lemmatize(sent[k][0], convert_tag(sent[k][1][0])) for k in range(len(sent))]

In [5]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

In [6]:
en_stop = set(get_stop_words('en'))

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        #print filename
        # read the whole file as lowercase string
        string = (f.read()).lower()
        try:
            temp = word_tokenize(string)
        except:
            string = string.decode("latin-1")
            temp = word_tokenize(string)
        
        tokens = []
        # tokenize that string
        for word in temp:
            w =  only_alphabet(word).lower()
            if w not in en_stop:
                if w:
                    tokens.append(w)
        tokens = tag_and_lem(tokens)
        yield tokens
        f.close()

        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [7]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('dict/lem10')

2018-05-04 15:56:59,521 : INFO : loading Dictionary object from dict/lem10
2018-05-04 15:56:59,574 : INFO : loaded dict/lem10


In [8]:
######################################## load BOW
corpus = corpora.MmCorpus('corpus/bow_lem10.mm')

2018-05-04 15:56:59,696 : INFO : loaded corpus index from corpus/bow_lem10.mm.index
2018-05-04 15:56:59,697 : INFO : initializing cython corpus reader from corpus/bow_lem10.mm
2018-05-04 15:56:59,698 : INFO : accepted corpus with 351985 documents, 169716 features, 99651009 non-zero entries


In [9]:
# %%time

# c = texts()
# count = 0
# for x in c.everything():
#     x = [ i for i in x if i in dictionary.token2id]
#     with open(os.path.normpath('Token_corpus2/' + str(count)), 'wb') as fp:
#         pickle.dump(x, fp)
#     count=count+1


In [10]:
def token_corpus(i):
    with open (os.path.normpath('Token_corpus2/' + str(i)), 'rb') as fp:
            itemlist = pickle.load(fp)
            return itemlist   

In [11]:
class iter_token_corpus:
    def __iter__(self):
        for i in xrange(0, 351985):
            with open (os.path.normpath('Token_corpus2/' + str(i)), 'rb') as fp:
                itemlist = pickle.load(fp)
                yield itemlist

In [12]:
%%time
model = gensim.models.Word2Vec(iter_token_corpus(), hs=1, size=300, iter = 20, min_count=0, sample=.00001, workers=8)

2018-05-04 15:57:00,789 : INFO : collecting all words and their counts
2018-05-04 15:57:00,791 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-04 15:57:12,350 : INFO : PROGRESS: at sentence #10000, processed 12250127 words, keeping 69492 word types
2018-05-04 15:57:23,128 : INFO : PROGRESS: at sentence #20000, processed 23019662 words, keeping 93100 word types
2018-05-04 15:57:34,982 : INFO : PROGRESS: at sentence #30000, processed 33121221 words, keeping 108174 word types
2018-05-04 15:57:43,926 : INFO : PROGRESS: at sentence #40000, processed 41771479 words, keeping 121160 word types
2018-05-04 15:57:50,668 : INFO : PROGRESS: at sentence #50000, processed 48600564 words, keeping 127318 word types
2018-05-04 15:57:57,038 : INFO : PROGRESS: at sentence #60000, processed 54839174 words, keeping 133012 word types
2018-05-04 15:58:02,473 : INFO : PROGRESS: at sentence #70000, processed 60283301 words, keeping 136653 word types
2018-05-04 15:58:08,987 : I

2018-05-04 16:03:07,666 : INFO : EPOCH 1 - PROGRESS: at 2.07% examples, 85243 words/s, in_qsize 0, out_qsize 1
2018-05-04 16:03:08,668 : INFO : EPOCH 1 - PROGRESS: at 2.09% examples, 85218 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:03:09,704 : INFO : EPOCH 1 - PROGRESS: at 2.13% examples, 84884 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:03:10,755 : INFO : EPOCH 1 - PROGRESS: at 2.20% examples, 84856 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:03:11,788 : INFO : EPOCH 1 - PROGRESS: at 2.26% examples, 84541 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:03:12,813 : INFO : EPOCH 1 - PROGRESS: at 2.38% examples, 84012 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:03:13,846 : INFO : EPOCH 1 - PROGRESS: at 2.46% examples, 83465 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:03:14,873 : INFO : EPOCH 1 - PROGRESS: at 2.52% examples, 82981 words/s, in_qsize 0, out_qsize 1
2018-05-04 16:03:15,883 : INFO : EPOCH 1 - PROGRESS: at 2.58% examples, 82608 words/s, in_qsize 0, out_qsize 0
2

2018-05-04 16:04:24,498 : INFO : EPOCH 1 - PROGRESS: at 5.17% examples, 57891 words/s, in_qsize 0, out_qsize 1
2018-05-04 16:04:25,494 : INFO : EPOCH 1 - PROGRESS: at 5.21% examples, 57791 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:04:26,545 : INFO : EPOCH 1 - PROGRESS: at 5.25% examples, 57645 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:04:27,592 : INFO : EPOCH 1 - PROGRESS: at 5.29% examples, 57515 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:04:28,614 : INFO : EPOCH 1 - PROGRESS: at 5.34% examples, 57358 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:04:29,614 : INFO : EPOCH 1 - PROGRESS: at 5.42% examples, 57253 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:04:30,626 : INFO : EPOCH 1 - PROGRESS: at 5.45% examples, 57109 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:04:31,659 : INFO : EPOCH 1 - PROGRESS: at 5.50% examples, 56907 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:04:32,678 : INFO : EPOCH 1 - PROGRESS: at 5.54% examples, 56762 words/s, in_qsize 0, out_qsize 0
2

2018-05-04 16:05:40,882 : INFO : EPOCH 1 - PROGRESS: at 8.45% examples, 51863 words/s, in_qsize 0, out_qsize 1
2018-05-04 16:05:41,907 : INFO : EPOCH 1 - PROGRESS: at 8.51% examples, 51834 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:05:42,932 : INFO : EPOCH 1 - PROGRESS: at 8.56% examples, 51794 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:05:44,007 : INFO : EPOCH 1 - PROGRESS: at 8.62% examples, 51765 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:05:45,124 : INFO : EPOCH 1 - PROGRESS: at 8.70% examples, 51700 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:05:46,148 : INFO : EPOCH 1 - PROGRESS: at 8.81% examples, 51704 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:05:47,230 : INFO : EPOCH 1 - PROGRESS: at 8.92% examples, 51674 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:05:48,266 : INFO : EPOCH 1 - PROGRESS: at 8.98% examples, 51650 words/s, in_qsize 1, out_qsize 0
2018-05-04 16:05:49,345 : INFO : EPOCH 1 - PROGRESS: at 9.03% examples, 51637 words/s, in_qsize 0, out_qsize 0
2

2018-05-04 16:06:58,206 : INFO : EPOCH 1 - PROGRESS: at 12.58% examples, 50540 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:06:59,234 : INFO : EPOCH 1 - PROGRESS: at 12.63% examples, 50526 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:07:00,252 : INFO : EPOCH 1 - PROGRESS: at 12.71% examples, 50517 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:07:01,290 : INFO : EPOCH 1 - PROGRESS: at 12.79% examples, 50509 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:07:02,340 : INFO : EPOCH 1 - PROGRESS: at 12.89% examples, 50511 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:07:03,351 : INFO : EPOCH 1 - PROGRESS: at 12.97% examples, 50501 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:07:04,473 : INFO : EPOCH 1 - PROGRESS: at 13.06% examples, 50446 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:07:05,484 : INFO : EPOCH 1 - PROGRESS: at 13.18% examples, 50457 words/s, in_qsize 0, out_qsize 0
2018-05-04 16:07:06,534 : INFO : EPOCH 1 - PROGRESS: at 13.25% examples, 50458 words/s, in_qsize 0, out_

...

2018-05-05 00:03:39,118 : INFO : EPOCH 20 - PROGRESS: at 91.67% examples, 58546 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:03:40,126 : INFO : EPOCH 20 - PROGRESS: at 91.69% examples, 58542 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:03:41,194 : INFO : EPOCH 20 - PROGRESS: at 91.71% examples, 58533 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:03:42,246 : INFO : EPOCH 20 - PROGRESS: at 91.72% examples, 58536 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:03:43,285 : INFO : EPOCH 20 - PROGRESS: at 91.74% examples, 58528 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:03:44,307 : INFO : EPOCH 20 - PROGRESS: at 91.76% examples, 58523 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:03:45,338 : INFO : EPOCH 20 - PROGRESS: at 91.79% examples, 58522 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:03:46,362 : INFO : EPOCH 20 - PROGRESS: at 91.82% examples, 58520 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:03:47,394 : INFO : EPOCH 20 - PROGRESS: at 91.85% examples, 58516 words/s, in_qsiz

2018-05-05 00:04:55,211 : INFO : EPOCH 20 - PROGRESS: at 93.44% examples, 58526 words/s, in_qsize 1, out_qsize 0
2018-05-05 00:04:56,282 : INFO : EPOCH 20 - PROGRESS: at 93.46% examples, 58528 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:04:57,398 : INFO : EPOCH 20 - PROGRESS: at 93.48% examples, 58529 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:04:58,406 : INFO : EPOCH 20 - PROGRESS: at 93.50% examples, 58531 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:04:59,446 : INFO : EPOCH 20 - PROGRESS: at 93.52% examples, 58539 words/s, in_qsize 0, out_qsize 1
2018-05-05 00:05:00,454 : INFO : EPOCH 20 - PROGRESS: at 93.53% examples, 58542 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:05:01,458 : INFO : EPOCH 20 - PROGRESS: at 93.55% examples, 58539 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:05:02,594 : INFO : EPOCH 20 - PROGRESS: at 93.57% examples, 58531 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:05:03,600 : INFO : EPOCH 20 - PROGRESS: at 93.59% examples, 58527 words/s, in_qsiz

2018-05-05 00:06:10,967 : INFO : EPOCH 20 - PROGRESS: at 95.52% examples, 58536 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:06:12,050 : INFO : EPOCH 20 - PROGRESS: at 95.58% examples, 58532 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:06:13,117 : INFO : EPOCH 20 - PROGRESS: at 95.63% examples, 58529 words/s, in_qsize 0, out_qsize 2
2018-05-05 00:06:14,146 : INFO : EPOCH 20 - PROGRESS: at 95.69% examples, 58530 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:06:15,161 : INFO : EPOCH 20 - PROGRESS: at 95.73% examples, 58528 words/s, in_qsize 1, out_qsize 0
2018-05-05 00:06:16,162 : INFO : EPOCH 20 - PROGRESS: at 95.78% examples, 58526 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:06:17,178 : INFO : EPOCH 20 - PROGRESS: at 95.83% examples, 58531 words/s, in_qsize 0, out_qsize 1
2018-05-05 00:06:18,178 : INFO : EPOCH 20 - PROGRESS: at 95.86% examples, 58527 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:06:19,182 : INFO : EPOCH 20 - PROGRESS: at 95.90% examples, 58528 words/s, in_qsiz

2018-05-05 00:07:26,483 : INFO : EPOCH 20 - PROGRESS: at 98.35% examples, 58425 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:07:27,488 : INFO : EPOCH 20 - PROGRESS: at 98.41% examples, 58426 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:07:28,527 : INFO : EPOCH 20 - PROGRESS: at 98.46% examples, 58429 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:07:29,533 : INFO : EPOCH 20 - PROGRESS: at 98.49% examples, 58433 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:07:30,550 : INFO : EPOCH 20 - PROGRESS: at 98.53% examples, 58423 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:07:31,561 : INFO : EPOCH 20 - PROGRESS: at 98.57% examples, 58418 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:07:32,661 : INFO : EPOCH 20 - PROGRESS: at 98.60% examples, 58411 words/s, in_qsize 0, out_qsize 0
2018-05-05 00:07:33,696 : INFO : EPOCH 20 - PROGRESS: at 98.62% examples, 58403 words/s, in_qsize 0, out_qsize 1
2018-05-05 00:07:34,714 : INFO : EPOCH 20 - PROGRESS: at 98.67% examples, 58405 words/s, in_qsiz

CPU times: user 9h 26min 6s, sys: 17min 5s, total: 9h 43min 11s
Wall time: 8h 11min 12s


In [13]:
model.save('py_models/W2V_CBOW')

2018-05-05 00:08:13,757 : INFO : saving Word2Vec object under py_models/W2V_CBOW, separately None
2018-05-05 00:08:13,790 : INFO : storing np array 'vectors' to py_models/W2V_CBOW.wv.vectors.npy
2018-05-05 00:08:14,033 : INFO : not storing attribute vectors_norm
2018-05-05 00:08:14,034 : INFO : storing np array 'syn1neg' to py_models/W2V_CBOW.trainables.syn1neg.npy
2018-05-05 00:08:14,182 : INFO : storing np array 'syn1' to py_models/W2V_CBOW.trainables.syn1.npy
2018-05-05 00:08:14,331 : INFO : not storing attribute cum_table
2018-05-05 00:08:21,751 : INFO : saved py_models/W2V_CBOW


In [14]:
model = gensim.models.Word2Vec.load('py_models/W2V_CBOW')

2018-05-05 00:08:21,766 : INFO : loading Word2Vec object from py_models/W2V_CBOW
2018-05-05 00:08:25,684 : INFO : loading vocabulary recursively from py_models/W2V_CBOW.vocabulary.* with mmap=None
2018-05-05 00:08:25,691 : INFO : loading wv recursively from py_models/W2V_CBOW.wv.* with mmap=None
2018-05-05 00:08:25,692 : INFO : loading vectors from py_models/W2V_CBOW.wv.vectors.npy with mmap=None
2018-05-05 00:08:27,208 : INFO : setting ignored attribute vectors_norm to None
2018-05-05 00:08:27,209 : INFO : loading trainables recursively from py_models/W2V_CBOW.trainables.* with mmap=None
2018-05-05 00:08:27,210 : INFO : loading syn1neg from py_models/W2V_CBOW.trainables.syn1neg.npy with mmap=None
2018-05-05 00:08:28,916 : INFO : loading syn1 from py_models/W2V_CBOW.trainables.syn1.npy with mmap=None
2018-05-05 00:08:30,699 : INFO : setting ignored attribute cum_table to None
2018-05-05 00:08:30,700 : INFO : loaded py_models/W2V_CBOW


In [15]:
# x.doesnt_match(words)

# #     Which word from the given list doesn’t go with the others?
# #     Parameters:	words – List of words
# #     Returns:	The word further away from the mean of all words.
# #     Return type:	str

# x.index2word
# ### list of words in the sorted order of their frequency 

# y = x.vocab['parti']
# y.count
# # find freq. of any word


# x.most_similar_cosmul(positive=None, negative=None, topn=10)
# # Find the top-N most similar words, using the multiplicative combination objective proposed by 
# # Omer Levy and Yoav Goldberg.Positive words still contribute positively towards the similarity, negative words 
# # negatively, but with less susceptibility to one large distance dominating the calculation.

# x.most_similar_to_given(entity1, entities_list)
# #    Return the entity from entities_list most similar to entity1.

# x.n_similarity(ws1, ws2)
# #Compute cosine similarity between two sets of words


# x.similar_by_vector(x['fit'], topn=10, restrict_vocab=None)
# # closest vectors
# # if topn is False, similar_by_vector returns the vector of similarity scores.

# x.similar_by_word(word, topn=10, restrict_vocab=None)
# #  Find the top-N most similar words.

# x.similarity(w1, w2)
# #Compute cosine similarity between two words.

# x.wmdistance(doc1, doc2)
# # word movers distance

# y = x.vocab['parti']
# y.count
# #freq in corpus

In [17]:
%%time

import math
from six import iteritems
from six.moves import xrange
idf = {}
for word, freq in dictionary.dfs.iteritems():
    idf[dictionary[word]] = math.log(dictionary.num_docs) - math.log(freq)


CPU times: user 336 ms, sys: 40 ms, total: 376 ms
Wall time: 293 ms


In [18]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [19]:
w2v = model.wv
def vec(document):
    return np.add.reduce([idf[i]*w2v[i] for i in document if i in dictionary.token2id])

In [20]:
%%time
doc_vec = []
for x in iter_token_corpus():
    x = vec(x)
    doc_vec.append(x)

CPU times: user 41min 17s, sys: 12.6 s, total: 41min 30s
Wall time: 58min 2s


In [21]:
from scipy.spatial.distance import cosine
def get_scores(query):
    scores = []
    i = 0
    query = vec(query)
    for x in doc_vec:
        if isinstance(x, np.ndarray):
            score = 1 - cosine(x, query)
        else:
            score = -1000
        scores.append(score)
    return scores

In [22]:
%%time
results = copy.deepcopy(query_truth)

sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    sims = get_scores(query)
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
        

CPU times: user 2min 36s, sys: 1.17 s, total: 2min 37s
Wall time: 2min 53s


In [23]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 0.44791001081466675, 2668],
  ['ConsumerCourt_DCDRC_106530', '0', 0.2213520109653473, 23720],
  ['ConsumerCourt_DCDRC_107608', '1', 0.5074297785758972, 1197],
  ['ConsumerCourt_DCDRC_114291', '1', 0.5309199690818787, 829],
  ['ConsumerCourt_DCDRC_114382', '1', 0.5046230554580688, 1254],
  ['ConsumerCourt_DCDRC_118185', '1', 0.5567267537117004, 459],
  ['ConsumerCourt_DCDRC_130318', '0', 0.551900327205658, 518],
  ['ConsumerCourt_DCDRC_130570', '1', 0.5862826704978943, 207],
  ['ConsumerCourt_DCDRC_131146', '1', 0.5727462768554688, 301],
  ['ConsumerCourt_DCDRC_131717', '1', 0.5598759055137634, 418],
  ['ConsumerCourt_DCDRC_131741', '1', 0.4994177222251892, 1375],
  ['ConsumerCourt_DCDRC_131818', '1', 0.5408750772476196, 667],
  ['ConsumerCourt_DCDRC_131950', '0', 0.4982038140296936, 1407],
  ['ConsumerCourt_DCDRC_131972', '0', 0.5886817574501038, 191],
  ['ConsumerCourt_DCDRC_132932', '0', 0.6350814700126648, 12],
  ['ConsumerCourt_DCDRC_13359

In [24]:
model_name = 'W2V_CBOW_IDF'

with open('py_results/' + model_name + 'sim_list.pkl', 'wb') as output:
    pickle.dump(sim_list, output, pickle.HIGHEST_PROTOCOL)
with open('py_results/' + model_name + 'rank_list.pkl', 'wb') as output:
    pickle.dump(rank_list, output, pickle.HIGHEST_PROTOCOL)


In [25]:
# Store results in 1st sheet


wb = Workbook()
dest_filename = 'result_excel/' + model_name + '.xlsx'
ws1 = wb.active
ws1.title = "ground_truth"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

#Query wise (F score)

import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean



places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name ]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name ]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = dest_filename)


#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean


places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name ]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name ]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = dest_filename)

In [27]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, index= False, header = False, **to_excel_kwargs)

    # save the workbook
    writer.save()

    
f = pd.ExcelFile(dest_filename)
sec_dest = 'result_excel/all_results.xlsx'
f2 = pd.ExcelFile(sec_dest)
append_df_to_excel(sec_dest, f.parse(1), sheet_name= f2.sheet_names[0])
append_df_to_excel(sec_dest, f.parse(2), sheet_name= f2.sheet_names[1])

In [ ]:
############################################################

In [ ]:
def Top_x(x, R):
    R = list(R)
    x_names = []
    for i in range(x):
        x_names.append(filenames[R.index(i + 1)].strip('.txt'))
    return x_names
    
    

In [ ]:
def score_stat(word, freq, index):
    index_doc = dict(bm25.corpus[index])
    word = dictionary.doc2bow([word])[0][0]
    idf = bm25.idf[word] if bm25.idf[word] >= 0 else EPSILON * average_idf
    score = freq*((idf * index_doc[word] * (PARAM_K1 + 1)
              / (index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / bm25.avgdl))))
    return (index_doc[word], score)

In [ ]:
wb = Workbook()
dest_filename = 'Temp.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)
def common_words(q, F):
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    index = filenames.index(F + '.txt')
    index_doc = dict(bm25.corpus[index])
    row = []
    for word, freq in  zip(q_words, q_freq):
        w = dictionary.doc2bow([word])[0][0]
        if w in index_doc: 
            f, s = score_stat(word, freq, index)
            row.append([word + ', ' + str(round(bm25.idf[w], 2)), freq, f,  round( (s*100)/sim_list[q][index], 1)])  
    row.sort(key = lambda x: (x[3],x[1],x[2]), reverse = True)
    wb = load_workbook('Temp.xlsx')
    ws = wb.create_sheet(title=str(q) + F)
    ws.append(['word,idf', 'Query freq', 'freq', '% score'])
    for i in row:
        ws.append(i)
    wb.save(filename = 'Temp.xlsx')


In [ ]:
f = open("toberead.txt", 'r') 
count = 0
q = 1
for i in f:
    if i == '\n':
        continue
    count = count + 1
    common_words(q, i.strip())
    if count % 5 == 0:
        q = q + 1


In [ ]:
for i in files_to_tokens(full_filenames[filenames.index(doc + '.txt')]):
    for j in i:
        print j

In [ ]:
doc = 'ConsumerCourt_SCDRC_30176'
corpus[]

In [ ]:
common_words(1, 'ConsumerCourt_DCDRC_41588')

In [ ]:
df = pd.DataFrame(index= None, columns=['words','pair'])


In [ ]:
df

In [ ]:
print df

In [ ]:
df.append(['lol', ('we','wew')])

In [ ]:
def uncommon_words(q, filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)

wb = load_workbook('Explore_BM25.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_uncom.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_uncom.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
#         for word, freq in  zip(q_words, q_freq):
#             if dictionary.doc2bow([word])[0][0] in index_doc: 
#                 f, s = score_stat(word, freq, index)
#                 row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
#             else:
#                 row.append('X')
        #ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_all.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_all.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(sec_row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)